## Imports

In [29]:
import os
import zipfile
import pandas
import requests

## Data acquisition

In [3]:
def get_items():
  folder_path = '../data/raw'
  processed_files = 0
  for file_name in os.listdir(folder_path):
    if not file_name.endswith('.zip'):
      continue
    zip_path = os.path.join(folder_path, file_name)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      file_items_name = f'{file_name.removesuffix('.zip')}_NotaFiscalItem.csv'
      with zip_ref.open(file_items_name) as file:
        file.seek(0)
        data_frame = pandas.read_csv(file, delimiter=';', encoding='latin1')
        data_frame.rename(columns={
          'CHAVE DE ACESSO': 'access_key',
          'DATA EMISSÃO': 'emission_date',
          'CPF/CNPJ Emitente': 'emission_owner',
          'NÚMERO PRODUTO': 'product_index',
          'DESCRIÇÃO DO PRODUTO/SERVIÇO': 'description',
          'CÓDIGO NCM/SH': 'ncm',
          'NCM/SH (TIPO DE PRODUTO)': 'ncm_description',
          'CFOP': 'cfop',
          'QUANTIDADE': 'quantity',
          'UNIDADE': 'unit_kind',
          'VALOR UNITÁRIO': 'unitary_value',
          'VALOR TOTAL': 'total_value',
        }, inplace=True)
        selected_fields = data_frame \
          .loc[:, [
            'access_key',
            'emission_date',
            'emission_owner',
            'product_index',
            'description',
            'ncm',
            'ncm_description',
            'cfop',
            'quantity',
            'unit_kind',
            'unitary_value',
            'total_value',
          ]]
        selected_fields['quantity'] = selected_fields['quantity'].str.replace(',', '.')
        selected_fields['total_value'] = selected_fields['total_value'].str.replace(',', '.')
        selected_fields['unitary_value'] = selected_fields['unitary_value'].str.replace(',', '.')
        yield selected_fields \
          .astype({
            'access_key': 'str',
            'emission_date': 'datetime64[ms]',
            'emission_owner': 'str',
            'product_index': 'int32',
            'description': 'str',
            'ncm': 'str',
            'ncm_description': 'str',
            'cfop': 'str',
            'quantity': 'float64',
            'unit_kind': 'str',
            'unitary_value': 'float64',
            'total_value': 'float64',
          })
        processed_files += 1
        print(f'Processed files {processed_files}')


def get_combined():
  combined_path = '../data/combined.parquet.br'
  if os.path.exists(combined_path):
    print('Reading cache combined data')
    return pandas.read_parquet(combined_path)
  combined_data = pandas.concat(get_items(), ignore_index=True)
  print('Start compression')
  combined_data.to_parquet(combined_path, index=False, compression='brotli')
  return combined_data

data_frame = get_combined()

Reading cache combined data


## Analyze dataset

In [7]:
total_rows = len(data_frame)
data_frame.info() 

data_frame.drop_duplicates(inplace=True)

unique_rows = len(data_frame)
removed_rows = total_rows - unique_rows
print(f'Total rows: {total_rows}')
print(f'Unique rows: {unique_rows}')
print(f'Removed rows: {removed_rows}')
print(f'Removed rows percentage: {round(removed_rows / total_rows, 4)}%')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17752229 entries, 0 to 17752228
Data columns (total 12 columns):
 #   Column           Dtype         
---  ------           -----         
 0   access_key       object        
 1   emission_date    datetime64[ms]
 2   emission_owner   object        
 3   product_index    int32         
 4   description      object        
 5   ncm              object        
 6   ncm_description  object        
 7   cfop             object        
 8   quantity         float64       
 9   unit_kind        object        
 10  unitary_value    float64       
 11  total_value      float64       
dtypes: datetime64[ms](1), float64(3), int32(1), object(7)
memory usage: 1.5+ GB
Total rows: 17752229
Unique rows: 17673249
Removed rows: 78980
Removed rows percentage: 0.0044%


In [22]:
table = dict(map(lambda x: (str(x), {'unique': 0}), range(0, 100)))
for table_item in data_frame['access_key'].unique():
  table[table_item[0:2]]['unique'] += 1
  
table = dict(filter(lambda x: x[1]['unique'] > 0, table.items()))
for key in table.keys():
  current_state = data_frame.where(data_frame['access_key'].str.startswith(key))
  total_value = current_state['total_value']
  table[key] |= {
    'total': total_value.count(),
    'total_value': round(total_value.sum(), 2),
  }

In [37]:
state_codes = dict(map(
  lambda x: (str(x['id']), x),
  requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados').json(),
))

In [43]:
meta_table_rows = list(table.items())
meta_table_rows.sort(key=lambda x: x[1]['total'], reverse=True)
for key, value in meta_table_rows:
  print('\t'.join([
    state_codes.get(key)['sigla'],
    state_codes.get(key)['nome'],
    "{:,}".format(value['unique']),
    "{:,}".format(value['total']),
    "{:,}".format(int(value['total_value'])),
  ]).replace(',', '.'))

SP	São Paulo	968.433	4.372.236	99.859.384.457
RJ	Rio de Janeiro	849.573	2.581.830	46.588.402.728
RS	Rio Grande do Sul	426.544	1.904.333	8.236.053.721
MG	Minas Gerais	353.555	1.342.135	18.058.849.286
DF	Distrito Federal	385.766	1.063.719	15.600.639.407
PR	Paraná	272.867	943.222	8.510.054.005
SC	Santa Catarina	223.651	575.369	5.383.060.717
PE	Pernambuco	147.843	510.864	6.940.477.382
BA	Bahia	112.289	442.067	1.592.486.551
MS	Mato Grosso do Sul	91.216	420.025	945.018.198
PA	Pará	101.956	407.554	1.028.692.878
AM	Amazonas	99.436	397.293	1.335.224.610
RN	Rio Grande do Norte	97.188	330.951	449.921.854
GO	Goiás	119.522	322.409	23.389.753.224
CE	Ceará	83.192	290.382	672.199.675
MA	Maranhão	72.172	287.946	381.705.137
ES	Espírito Santo	89.008	250.336	3.452.116.826
MT	Mato Grosso	106.433	247.252	490.956.647
PB	Paraíba	59.998	173.599	463.042.991
PI	Piauí	35.788	170.278	220.885.632
RO	Rondônia	48.759	156.884	371.488.485
RR	Roraima	26.410	121.510	598.998.213
AC	Acre	22.273	87.804	132.491.524
AL	Alagoa

## Pre processing